In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


# import class from CRUD Python module
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# instantiate class
username = "aacuser"
password = "1234"
shelter = AnimalShelter()


# read data into dataframe
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #unique identifier
    html.Center(html.B(html.H1(['Aaron Fehir', html.Br(),'SNHU CS-340 Dashboard']))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Set up features for interactive data table
        page_size=100,
        style_table={'height':'300px','overflowY':'auto','overflowX':'auto'},
        style_header={
            'backgroundColor':'rgb(230,230,230)',
            'fontWeight':'bold'        
        },
        style_data={
            'whiteSpace':'normal',
            'height':'auto'
        },
        
        #tooltips
        tooltip ={i: {
        'value': i,
        'use_with': 'both'  # both refers to header & data cell
    } for i in df.columns},
        tooltip_delay=0,
        tooltip_duration = None,
        
        #sorting features
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=False,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
        
    ),
    html.Br(),
    html.Hr(),
    #Arrange dashboard so that chart and geolocation chart appear side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    ])
      
   

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app